---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    # 1. create dataframe from txt file
    university_towns = pd.read_table('university_towns.txt', header = None)
    # explicitly display the values of colums
    pd.options.display.max_colwidth = 200
    # 2. Create two columns: 'State' and 'RegionName' 
    # First, create 'State' and 'RegionName' columns where the values are identical to original dataframe values. 
    university_towns['RegionName'] = university_towns[0]
    university_towns = university_towns.rename(columns = {0: 'State'})
    # Similarly, create 'RegionName' column where its values are identical copies of the original dataframe
    # 1) For 'State', change non-state values to None. Then use ffill method to replace them with corresponding state values
    # methods to use: replace, ffill
    # e.g method is replace) x = pd.DataFrame({'a': [1,2,3], 'b': [4,1,6], 'c':[7,8,9]})
    # x['a'] = x['a'].replace(to_replace = 1, value = 'apple', inplace = False)
    university_towns['State'] = university_towns['State'].replace(to_replace = [r"\(.*"], value =  [None], regex = True)
    university_towns['State'] = university_towns['State'].replace(to_replace = [r":"], value =  [None], regex = True)
    university_towns['State'] = university_towns['State'].replace(to_replace = [r","], value =  [None], regex = True)
    # e.g. method is ffill) x = pd.DataFrame({'a': [1,2,3], 'b': [4,1,6], 'c':[7,8,9]})
    # x['a'] = x['a'].replace(to_replace = [1,3], value = [None, None], inplace = False)
    # x['a'] = x['a'].ffill()
    university_towns['State'] = university_towns['State'].ffill()
    # 2) For 'RegionName', change non-region values (a.k.a state values) to None. 
    university_towns['RegionName'] = university_towns['RegionName'].replace(to_replace = [r"\[edit\]"], value =  [None], regex = True)
    # Now, Remove all rows that contain None value.
    university_towns = university_towns.dropna()
    # 3. For "State", removing characters from "[" to the end.
    university_towns['State'] = university_towns['State'].replace(to_replace = [r"\[.*"], value =  [''], regex = True)
    # 4. For "RegionName", when applicable, removing every character from " (" to the end.
    university_towns['RegionName'] = university_towns['RegionName'].replace(to_replace = [r"\ \(.*"], value =  [''], regex = True)
    # 5. Depending on how you read the data, you may need to remove newline character '\n'. 
    # '\n' in university_towns.values 
    # no need to do this part of cleaning
    answer = university_towns
    return answer

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    # From Bureau of Economic Analysis, US Department of Commerce, 
    # the GDP over time of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, 
    # in the file gdplev.xls. For this assignment, only look at GDP data from the first quarter of 2000 onward.

    gdp_over_time = pd.read_excel('gdplev.xls')
    pd.set_option('display.max_rows', 70)

    # drop useless columns, and rename columns to categorical description of their values
    gdp_over_time = gdp_over_time.drop(['Unnamed: 3', gdp_over_time.columns[7]], axis = 1)
    gdp_over_time = gdp_over_time.rename(columns = {'Unnamed: 1': gdp_over_time['Unnamed: 1'][4], 
                                                'Unnamed: 2': 'x',
                                                'Unnamed: 4': gdp_over_time['Unnamed: 4'][1],
                                                'Unnamed: 5': gdp_over_time['Unnamed: 5'][4],
                                                'Unnamed: 6': gdp_over_time['Unnamed: 6'][4],
                                                'Current-Dollar and "Real" Gross Domestic Product': 'Annual'
                                                })
    gdp_over_time = gdp_over_time.drop(['GDP in billions of current dollars'], axis = 1)
    gdp_over_time = gdp_over_time.rename(columns = {'Annual': 'Year', 'x': 'Annual GDP in billions of chained 2009 dollars', 
                                                'Quarterly': 'Quarter', 
                                                'GDP in billions of chained 2009 dollars': 'Quarterly GDP in billions of chained 2009 dollars'
                                                })
    # remove header
    gdp_over_time = gdp_over_time.drop(gdp_over_time.index[0:7])
    # only look at GDP data from the first quarter of 2000 onward. I don't need the first two columns (Year and Annual GDP) at all. 
    quarterly_gdp_over_time = gdp_over_time[['Quarter', 'Quarterly GDP in billions of chained 2009 dollars']]
    quarterly_gdp_over_time.index = pd.RangeIndex(start = 0, step =1, stop = 278)
    quarterly_gdp_over_time = quarterly_gdp_over_time.iloc[212:]
    quarterly_gdp_over_time.index = pd.RangeIndex(start = 0, step =1, stop = 66)
    # Find the recession period 
    # Use boolean mask to note GDP decline. False means decline, True means GDP increase or otherwise. 
    quarterly_gdp_over_time['GDP diff'] = (quarterly_gdp_over_time['Quarterly GDP in billions of chained 2009 dollars']
                                       .diff())
    quarterly_gdp_over_time['GDP diff'] = (quarterly_gdp_over_time['GDP diff']
                                       .apply(lambda x: False if x < 0 else True))
    # Find dataframe slice where the starting point is the earliest occurence of two consecutive False in 'GDP diff'
    # First create a booleabn column 'GDP diff' where True if GDP difference is positive and False if GDP difference is negative
    candidates_of_start_of_unique_recessions = []
    for x in quarterly_gdp_over_time.index:
        if (quarterly_gdp_over_time['GDP diff'][x] == False) and (quarterly_gdp_over_time['GDP diff'][x+1] == False):
            candidates_of_start_of_unique_recessions.append(quarterly_gdp_over_time['Quarter'][x])
    # recession start candidiates as a list of index
    candidates_of_start_of_unique_recessions_index = []
    for candidate in candidates_of_start_of_unique_recessions:
        candidates_of_start_of_unique_recessions_index.append(quarterly_gdp_over_time.index[quarterly_gdp_over_time['Quarter'] 
                                                                                        == candidate][0])
    # recession candidates as a list of tuples where each tuple represents a recession of this form (recession start: recession end)
    recession_candidates = []
    for candidate_index in candidates_of_start_of_unique_recessions_index:
        for index in quarterly_gdp_over_time.index[candidate_index:].tolist():
            if (quarterly_gdp_over_time['GDP diff'][index] == True) and (quarterly_gdp_over_time['GDP diff'][index + 1] == True):
                recession_candidates.append((candidate_index, index+1))
                break
    # start out with recessions as recession_candidates. 
    recessions = recession_candidates
    # non-recessions deduced from candidates given that they their start value falls the range of other recession candidates
    non_recessions = []
    for candidate in recession_candidates:
        for element in recession_candidates:
            if candidate != element:
                if candidate[0] in range(element[0], element[1]):
                    non_recessions.append(candidate)
            else:
                pass
    # remove non-recession from recession-candidates to get unique recessions
    for element in non_recessions:
        if element in recessions:
            recessions.remove(element)
    unique_recession = recessions[0]
    # recession start 
    answer = quarterly_gdp_over_time['Quarter'][unique_recession[0]]
    return answer

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp_over_time = pd.read_excel('gdplev.xls')
    pd.set_option('display.max_rows', 70)
    gdp_over_time = gdp_over_time.drop(['Unnamed: 3', gdp_over_time.columns[7]], axis = 1)
    gdp_over_time = gdp_over_time.rename(columns = {'Unnamed: 1': gdp_over_time['Unnamed: 1'][4], 
                                                'Unnamed: 2': 'x',
                                                'Unnamed: 4': gdp_over_time['Unnamed: 4'][1],
                                                'Unnamed: 5': gdp_over_time['Unnamed: 5'][4],
                                                'Unnamed: 6': gdp_over_time['Unnamed: 6'][4],
                                                'Current-Dollar and "Real" Gross Domestic Product': 'Annual'
                                                })
    gdp_over_time = gdp_over_time.drop(['GDP in billions of current dollars'], axis = 1)
    gdp_over_time = gdp_over_time.rename(columns = {'Annual': 'Year', 'x': 'Annual GDP in billions of chained 2009 dollars', 
                                                'Quarterly': 'Quarter', 
                                                'GDP in billions of chained 2009 dollars': 'Quarterly GDP in billions of chained 2009 dollars'
                                                })
    gdp_over_time = gdp_over_time.drop(gdp_over_time.index[0:7])
    quarterly_gdp_over_time = gdp_over_time[['Quarter', 'Quarterly GDP in billions of chained 2009 dollars']]
    quarterly_gdp_over_time.index = pd.RangeIndex(start = 0, step =1, stop = 278)
    quarterly_gdp_over_time = quarterly_gdp_over_time.iloc[212:]
    quarterly_gdp_over_time.index = pd.RangeIndex(start = 0, step =1, stop = 66)
    quarterly_gdp_over_time['GDP diff'] = (quarterly_gdp_over_time['Quarterly GDP in billions of chained 2009 dollars']
                                       .diff())
    quarterly_gdp_over_time['GDP diff'] = (quarterly_gdp_over_time['GDP diff']
                                       .apply(lambda x: False if x < 0 else True))
    candidates_of_start_of_unique_recessions = []
    for x in quarterly_gdp_over_time.index:
        if (quarterly_gdp_over_time['GDP diff'][x] == False) and (quarterly_gdp_over_time['GDP diff'][x+1] == False):
            candidates_of_start_of_unique_recessions.append(quarterly_gdp_over_time['Quarter'][x])
    candidates_of_start_of_unique_recessions_index = []
    for candidate in candidates_of_start_of_unique_recessions:
        candidates_of_start_of_unique_recessions_index.append(quarterly_gdp_over_time.index[quarterly_gdp_over_time['Quarter'] 
                                                                                        == candidate][0])
    recession_candidates = []
    for candidate_index in candidates_of_start_of_unique_recessions_index:
        for index in quarterly_gdp_over_time.index[candidate_index:].tolist():
            if (quarterly_gdp_over_time['GDP diff'][index] == True) and (quarterly_gdp_over_time['GDP diff'][index + 1] == True):
                recession_candidates.append((candidate_index, index+1))
                break
    recessions = recession_candidates
    non_recessions = []
    for candidate in recession_candidates:
        for element in recession_candidates:
            if candidate != element:
                if candidate[0] in range(element[0], element[1]):
                    non_recessions.append(candidate)
            else:
                pass
    for element in non_recessions:
        if element in recessions:
            recessions.remove(element)
    unique_recession = recessions[0]
    # recession end 
    answer = quarterly_gdp_over_time['Quarter'][unique_recession[1]]
    return answer

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    gdp_over_time = pd.read_excel('gdplev.xls')
    pd.set_option('display.max_rows', 70)
    gdp_over_time = gdp_over_time.drop(['Unnamed: 3', gdp_over_time.columns[7]], axis = 1)
    gdp_over_time = gdp_over_time.rename(columns = {'Unnamed: 1': gdp_over_time['Unnamed: 1'][4], 
                                                'Unnamed: 2': 'x',
                                                'Unnamed: 4': gdp_over_time['Unnamed: 4'][1],
                                                'Unnamed: 5': gdp_over_time['Unnamed: 5'][4],
                                                'Unnamed: 6': gdp_over_time['Unnamed: 6'][4],
                                                'Current-Dollar and "Real" Gross Domestic Product': 'Annual'
                                                })
    gdp_over_time = gdp_over_time.drop(['GDP in billions of current dollars'], axis = 1)
    gdp_over_time = gdp_over_time.rename(columns = {'Annual': 'Year', 'x': 'Annual GDP in billions of chained 2009 dollars', 
                                                'Quarterly': 'Quarter', 
                                                'GDP in billions of chained 2009 dollars': 
                                                'Quarterly GDP in billions of chained 2009 dollars'
                                                })
    gdp_over_time = gdp_over_time.drop(gdp_over_time.index[0:7])
    quarterly_gdp_over_time = gdp_over_time[['Quarter', 'Quarterly GDP in billions of chained 2009 dollars']]
    quarterly_gdp_over_time.index = pd.RangeIndex(start = 0, step =1, stop = 278)
    quarterly_gdp_over_time = quarterly_gdp_over_time.iloc[212:]
    quarterly_gdp_over_time.index = pd.RangeIndex(start = 0, step =1, stop = 66)
    quarterly_gdp_over_time['GDP diff'] = (quarterly_gdp_over_time['Quarterly GDP in billions of chained 2009 dollars']
                                       .diff())
    quarterly_gdp_over_time['GDP diff'] = (quarterly_gdp_over_time['GDP diff']
                                       .apply(lambda x: False if x < 0 else True))
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_start_index = quarterly_gdp_over_time[quarterly_gdp_over_time['Quarter'] == recession_start].index[0]
    recession_end_index = quarterly_gdp_over_time[quarterly_gdp_over_time['Quarter'] == recession_end].index[0]
    recession_df = quarterly_gdp_over_time.iloc[recession_start_index:recession_end_index]
    recession_bottom_gdp = recession_df['Quarterly GDP in billions of chained 2009 dollars'].min()
    answer = (recession_df['Quarter'][recession_df['Quarterly GDP in billions of chained 2009 dollars'] == recession_bottom_gdp]
          .values[0])
    return answer

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    pd.set_option('display.max_columns', 67)
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    # First, change the dataframe to multi-index of ["State","RegionName"]. I.e. change the columns ["State","RegionName"] to
    # A multilevel index. 
    #housing_data = housing_data.sort_values('State', ascending = True)
    housing_data = housing_data.set_index(["State","RegionName"])
    # Second, create new columns [2000q1,..., 2016q3] where each of these columns are quarters, consisting of the mean values of 
    # all months contained in the quarter. (* this is perhaps the hardest part to code)
    import re
    for column in housing_data:
        if (re.match(r'\d{4}-\d{2}', column)) and (column[0] == '2'): #this matches columns of string pattern 2XXX-XX
            if column[6] == '1':
                first_quarter_name = column[0:4] + 'q1'
                housing_data[first_quarter_name] = housing_data[[column, column[0:4]+'-'+'02', column[0:4]+'-'+'03']].mean(axis =1)
            if column[6] == '4':
                second_quarter_name = column[0:4] + 'q2'
                housing_data[second_quarter_name] = housing_data[[column, column[0:4]+'-'+'05', column[0:4]+'-'+'06']].mean(axis =1)
            if column[6] == '7':
                if (column[0:4]+'-'+'09') in housing_data:
                    third_quarter_name = column[0:4] + 'q3'
                    housing_data[third_quarter_name] = (housing_data[[column, column[0:4]+'-'+'08', column[0:4]+'-'+'09']]
                                                    .mean(axis =1))
                if (column[0:4]+'-'+'09') not in housing_data:
                    third_quarter_name = column[0:4] + 'q3'
                    housing_data[third_quarter_name] = (housing_data[[column, column[0:4]+'-'+'08']]
                                                    .mean(axis =1))
            if column[5:7] == '10':
                fourth_quarter_name = column[0:4] + 'q4'
                housing_data[fourth_quarter_name] = housing_data[[column, column[0:4]+'-'+'11', column[0:4]+'-'+'11']].mean(axis =1)
        else:
            pass
    # Third, remove all columns of this string pattern [YYYY-MM]
    cols = [column for column in housing_data if (not re.match(r'\d{4}-\d{2}', column))]
    only_columns = cols[4:]
    housing_data = housing_data[only_columns]
    # Fourth, change the state names from abbreviations to their full names
    housing_data.rename(index = states, inplace = True)
    answer = housing_data
    return answer

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    #1: create new data showing the decline/growth of housing prices between recession start and recession bottom. 
    # take convert_housing_data_to_quarters() dataframe, and select its subset in terms of columns ranging from recession start 
    # and recession bottom
    recession_start_column_index = convert_housing_data_to_quarters().columns.get_loc(get_recession_start())
    recession_bottom_column_index = convert_housing_data_to_quarters().columns.get_loc(get_recession_bottom())
    housing_price_during_recession= convert_housing_data_to_quarters()[convert_housing_data_to_quarters()
                                                   .columns[recession_start_column_index -1 : recession_bottom_column_index + 1]]
    
    #2: Establish constant terms (string values or column names) for recession start, recession bottom, or quarter before recession 
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    quarter_before_recession = '2008q2'
    
    #3: Merge university towns dataframe which has a boolean column all True, with housing_price_during_recession dataframe
    university_towns = get_list_of_university_towns()
    university_towns['is_university_town'] = True
    df = pd.merge(housing_price_during_recession, university_towns, how='outer', left_index=True, right_on=['State', 'RegionName'])
    df['is_university_town'] = df['is_university_town'].replace({np.NaN: False})
    
    #4: add price ratio column given by following formula: price_ratio=quarter_before_recession/recession_bottom
    df['price_ratio'] = df[quarter_before_recession]/df[recession_bottom]
    
    # 5: get p value by runing a t-test which is a function which returns p value on two groups, 
    # where the two groups are represented by two series,
    # or two columns of a dataframe (i.e. two groups to which their objects are each ascribed the price_ratio value)
    university_town_group = df[df['is_university_town'] == True]['price_ratio']
    non_university_town_group = df[df['is_university_town'] == False]['price_ratio']
    st, p = ttest_ind(university_town_group, non_university_town_group, nan_policy='omit',)
    
    # 6: get different. different = True if the t-test is True at a p<0.01 (we reject the null hypothesis), 
    #or different=False if otherwise (we cannot reject the null hypothesis).
    different = True
    if p < 0.01:
        pass
    else:
        different = False
        
    # 7: get better. The value for better should be either "university town" or "non-university town"
    # depending on which has a lower mean price ratio (which is equivilent to a
    # reduced market loss).'''
    university_town_group_mean_price_ratio = university_town_group.mean()
    non_university_town_group_mean_price_ratio = non_university_town_group.mean()
    better = 'non-university town'
    if university_town_group_mean_price_ratio < non_university_town_group_mean_price_ratio:
        better = 'university town'
    else:
        better = 'non-university town'   
    return (different, p, better)